In [1]:
import os
import json
import pandas as pd
import traceback

In [32]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
from langchain.chains import LLMChain

In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
KEY = os.environ["OCTOAI_API_TOKEN"]
os.environ["ENDPOINT_URL"] = "https://text.octoai.run/v1/chat/completions"

In [38]:
llm = OctoAIEndpoint(
    model_kwargs={
        "model": "llama-2-13b-chat-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.5,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant. Keep your responses limited to one short paragraph if possible.",
            },
        ],
    },
)

In [40]:
from langchain.llms import octoai_endpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [41]:
RESPONSE_JSON = {
    "1":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
    "2":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
    "3":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
    "4":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
    "5":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
    "6":{
        "mcq":"Multiple choice question",
        "options":{
            "a":"Choice here",
            "b":"Choice here",
            "c":"Choice here",
            "d":"Choice here",
        },
        "correct answer":"Correct Answer"
    },
}

In [42]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text is your job to
create a quiz of {number} multiple choice questions for {subject} student in an tone of {tone}
make sure that question is not repeating and check all the question confirming the text as well
make sure format your response like RESPONSE_JSON and use it as a guide.
Ensure to make {number} MCQS
###RESPONSE_JSON
{response_json}
"""

In [43]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [50]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generator_prompt, output_key="quiz" ,verbose=True)

In [46]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students
you need to evalute the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the complexity
if the quiz is not at per cognitive and analytical ablilities of the students,
update the quiz questions which needs to be changed and change the tone that it perfectly fits the student abilities
Quiz_Mcqs
{quiz}

Check from the expert english writer from the above quiz
"""

In [47]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [48]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review" ,verbose=True)

In [51]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], verbose=True, input_variables=["text","number","subject","tone","response_json"], output_variables=["quiz","review"])

In [52]:
file_path=r"D:\python\mcqGenerator\data.txt"

In [53]:
with open(file_path,'r') as file:
    Text = file.read()

In [58]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct answer": "Correct Answer"}, "2": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct answer": "Correct Answer"}, "3": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct answer": "Correct Answer"}, "4": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct answer": "Correct Answer"}, "5": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct answer": "Correct Answer"}, "6": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct 

In [88]:
input_variables = {
    "text": "What do you think about SEO",
    "number": 4,
    "subject": "Artificial Intelligence",
    "tone": "Simple",
    "response_json":RESPONSE_JSON
}

In [89]:
result = quiz_chain.run(input_variables)
print(result)



> Entering new LLMChain chain...
Prompt after formatting:

Text:What do you think about SEO
You are an expert MCQ maker. Given the above text is your job to
create a quiz of 4 multiple choice questions for Artificial Intelligence student in an tone of Simple
make sure that question is not repeating and check all the question confirming the text as well
make sure format your response like RESPONSE_JSON and use it as a guide.
Ensure to make 4 MCQS
###RESPONSE_JSON
{'1': {'mcq': 'Multiple choice question', 'options': {'a': 'Choice here', 'b': 'Choice here', 'c': 'Choice here', 'd': 'Choice here'}, 'correct answer': 'Correct Answer'}, '2': {'mcq': 'Multiple choice question', 'options': {'a': 'Choice here', 'b': 'Choice here', 'c': 'Choice here', 'd': 'Choice here'}, 'correct answer': 'Correct Answer'}, '3': {'mcq': 'Multiple choice question', 'options': {'a': 'Choice here', 'b': 'Choice here', 'c': 'Choice here', 'd': 'Choice here'}, 'correct answer': 'Correct Answer'}, '4': {'mcq': 'Mul

In [87]:
json.loads(result)

JSONDecodeError: Expecting value: line 1 column 3 (char 2)